In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from sklearn.metrics import f1_score

In [2]:
import sys
sys.path.append('ml_lib/')
from encoding import FreqeuncyEncoding
from custom_classifier_mutliclass import Estimator
from hyperopt_multiclass import HyperOptModelSelection
from hyperopt import hp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [3]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle('bureau_future_feats_corrected.pkl')
# bur_df2 = pd.read_pickle('bureau_loan_duration_hist_str_feats.pkl')
bur_df2 = pd.read_pickle('ltfs_bureau_future_feats.pkl')
bur_df3 = pd.read_pickle('bur_future_feats.pkl')
# bur_df3 = pd.read_pickle('future_from_disb_date_feats.pkl')
# bur_df4 = pd.read_pickle('gold_trac_future_from_disb_date_feats.pkl')
# bur_df5 = pd.read_csv('leak_feature.csv')


In [5]:
bur_df2.shape,bur_df3.shape

((137534, 120), (143400, 1036))

In [6]:
target_map = {'No Top-up Service': 0,
 '12-18 Months': 1,
 '18-24 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '36-48 Months': 5,
 ' > 48 Months': 6,
 }
train['Top-up Month'] = train['Top-up Month'].map(target_map)

In [7]:
reverse_map = {v:k for k,v in target_map.items()}

In [8]:
reverse_map

{0: 'No Top-up Service',
 1: '12-18 Months',
 2: '18-24 Months',
 3: '24-30 Months',
 4: '30-36 Months',
 5: '36-48 Months',
 6: ' > 48 Months'}

In [9]:
train['Top-up Month'].value_counts()

0    106677
6      8366
5      3656
3      3492
4      3062
2      2368
1      1034
Name: Top-up Month, dtype: int64

In [10]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [11]:
bur_df.shape

(139964, 112)

In [12]:
df = df.merge(bur_df,on='ID',how='left').merge(bur_df2,on='ID',how='left').merge(bur_df3,on='ID',how='left')
# df = df.merge(bur_df,on='ID',how='left').merge(bur_df2,on='ID',how='left').merge(bur_df3,
#                                  on='ID',how='left').merge(bur_df4,on='ID',how='left').merge(bur_df5,on='ID',how='left')
# df = df.merge(bur_df,on='ID',how='left')
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])
df['MaturityDAte'] = pd.to_datetime(df['MaturityDAte'])

In [13]:
df.shape

(143400, 1291)

In [14]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [15]:
df['pin1'] = df['ZiPCODE'].fillna('000000').astype('str').str[:2]
df['pin2'] = df['ZiPCODE'].fillna('000000').astype('str').str[2:4]
df['pin3'] = df['ZiPCODE'].fillna('000000').astype('str').str[4:6:]
df['cal_tenor'] = (df['MaturityDAte'] - df['DisbursalDate']).dt.days
df['emi_sal_ratio'] = df['EMI']/df['MonthlyIncome']
df['ltv2'] = df['DisbursalAmount']/df['AssetCost']

In [16]:
df['disb_year'] = df['DisbursalDate'].dt.year
df['disb_mon'] = df['DisbursalDate'].dt.month
df['disb_day'] = df['DisbursalDate'].dt.day
df['disb_dow'] = df['DisbursalDate'].dt.dayofweek

df['mat_year'] = df['MaturityDAte'].dt.year
df['mat_mon'] = df['MaturityDAte'].dt.month
df['mat_day'] = df['MaturityDAte'].dt.day
df['mat_dow'] = df['MaturityDAte'].dt.dayofweek

In [17]:
cat_cols = ['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'Area','ManufacturerID',
            'SupplierID','pin1','pin2','pin3', 'SEX', 'City', 'State','BranchID',
           ]
target = 'Top-up Month'
drop_cols = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','ZiPCODE']
num_cols = (df.columns[~df.columns.isin([target]+drop_cols+cat_cols)].tolist())
use_cols = cat_cols+num_cols

In [18]:
# df.columns[df.columns.str.contains('is_ltfs')]

In [19]:
len(use_cols)

1298

In [20]:
fe = FreqeuncyEncoding(categorical_columns=cat_cols,normalize=True,return_df=True)
df = fe.fit_transform(df)

In [21]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [22]:
train = df[df.ID.isin(train.ID)]
test = df[df.ID.isin(test.ID)]
train = train.sort_values('ID',ascending=True)
test = test.sort_values('ID',ascending=True)
train[target] = train[target].astype('int')

In [23]:
train.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [24]:
folds = StratifiedKFold(5, shuffle = True, random_state = 2)
folds = [(x,y) for x,y in folds.split(train,train[target])]

In [25]:
len(use_cols)

1298

In [26]:
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 32, 192, 32),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.5, 1, 0.1),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.1),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}

In [27]:
est = Estimator(model = LGBMClassifier(),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [28]:
%%time
hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp3.txt')

CPU times: user 214 µs, sys: 76 µs, total: 290 µs
Wall time: 213 µs


In [28]:
# %%time
# hyp.fit(train[use_cols].values,train[target].values)

In [31]:
# params={'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 40.0, 'n_estimators': 5000,
#         'n_jobs': -1, 'num_leaves': 96, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

params = {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.05, 'min_child_weight': 40.0, 'n_estimators': 5000,
          'n_jobs': -1, 'num_leaves': 192, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5} ### cv0.6225

In [29]:
est = Estimator(model = LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,learning_rate=0.1,
               n_estimators=5000, num_leaves=64, objective='multiclass',
               subsample_freq=5),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

# est = Estimator(model = LGBMClassifier(**params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [30]:
temp = est.fit_transform(train[use_cols].values,train[target].values)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.287016	valid_1's multi_logloss: 0.211526
[200]	valid_0's multi_logloss: 0.288224	valid_1's multi_logloss: 0.162028
Early stopping, best iteration is:
[123]	valid_0's multi_logloss: 0.286587	valid_1's multi_logloss: 0.19784
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.282194	valid_1's multi_logloss: 0.212639
[200]	valid_0's multi_logloss: 0.282802	valid_1's multi_logloss: 0.163004
Early stopping, best iteration is:
[120]	valid_0's multi_logloss: 0.281553	valid_1's multi_logloss: 0.200533
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.28294	valid_1's multi_logloss: 0.211672
[200]	valid_0's multi_logloss: 0.284252	valid_1's multi_logloss: 0.162387
Early stopping, best iteration is:
[120]	valid_0's multi_logloss: 0.282382	valid_1's multi_logloss: 0.199647
Training until validation scores don't improv

In [31]:
est.cv_scores,est.overall_cv_score

([0.7036984062889958,
  0.7107815234666283,
  0.7133829654890139,
  0.7151991679799308,
  0.7216213057733983],
 0.7130629870474838)

In [32]:
feat_imp = est.feature_importance_df(use_cols)

In [33]:
feat_imp.head(50)

,column,feature_importance,rank
21,MonthlyIncome,0.023151,1
12,State,0.017776,2
1287,cal_tenor,0.017119,3
1288,emi_sal_ratio,0.015735,4
11,City,0.015635,5
1290,disb_year,0.015396,6
6,SupplierID,0.014700,7
20,AGE,0.014146,8
13,BranchID,0.014048,9
7,pin1,0.013400,10


In [34]:
# for i in [100,300,500,700,900]:
#     col = feat_imp.iloc[:i]['column']
#     temp = est.fit_transform(train[col].values,train[target].values)
#     print(est.overall_cv_score)

In [35]:
pd.Series(temp.argmax(axis=1)).value_counts(1)

0    0.887163
6    0.045517
5    0.018398
3    0.015911
4    0.014271
2    0.012615
1    0.006125
dtype: float64

In [36]:
f1_score(train[target],pd.Series(temp.argmax(axis=1)),average='macro')

0.7130629870474838

In [37]:
test_preds = [est.predict_proba(test[use_cols].values) for est in est.fitted_models]

In [38]:
test['Top-up Month'] = np.sum(test_preds,axis=0).argmax(axis=1)

In [39]:
test['Top-up Month'].value_counts(1)

0    0.917735
6    0.038182
5    0.017362
4    0.008545
2    0.006646
3    0.006307
1    0.005222
Name: Top-up Month, dtype: float64

In [38]:
# test['Top-up Month'].value_counts(1)

In [36]:
sub = test[['ID','Top-up Month']]

In [37]:
sub['Top-up Month'] = sub['Top-up Month'].map(reverse_map)

In [38]:
sub.to_csv('LGBM_CV0.713.csv',index=False)